In [16]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline
from tqdm.auto import tqdm
import concurrent.futures
from multiprocessing import Pool
import copy,os,sys
from collections import Counter,deque
import matplotlib.pyplot as plt
import json
import functools
import itertools
import math

In [2]:
import torch

In [40]:
nwords=10
emb_size=3
# input matrix
emb = torch.nn.Embedding(nwords,emb_size)
torch.nn.init.uniform_(emb.weight,-0.25,0.25)
# output matrix
projection = torch.nn.Linear(emb_size, nwords)
torch.nn.init.uniform_(projection.weight, -0.25, 0.25)

Parameter containing:
tensor([[-0.2176,  0.2311, -0.1036],
        [ 0.0611,  0.1926, -0.2174],
        [ 0.2406,  0.1752,  0.0449],
        [-0.1772, -0.2120, -0.0990],
        [-0.0699, -0.1236, -0.1812],
        [-0.1649,  0.1863, -0.1928],
        [ 0.2110, -0.0783,  0.0201],
        [ 0.0808,  0.0801,  0.0208],
        [ 0.0301, -0.2290, -0.0842],
        [-0.2400, -0.0680,  0.0410]], requires_grad=True)

Parameter containing:
tensor([[-0.0676, -0.1233, -0.1751],
        [-0.1638,  0.0504, -0.2271],
        [-0.0458,  0.2188,  0.2253],
        [ 0.1069, -0.2410,  0.2020],
        [ 0.1209,  0.2178, -0.0388],
        [ 0.1979, -0.0137, -0.1540],
        [-0.1406,  0.0408, -0.2246],
        [ 0.1237,  0.1747, -0.0564],
        [ 0.1977,  0.1329,  0.1071],
        [ 0.0365, -0.1141,  0.1727]], requires_grad=True)

In [51]:
words=torch.LongTensor([0,1])
emb_res = emb(words)
emb_sum = torch.sum(emb_res, dim=0) # emb_size
# emb_sum = emb_sum.view(1, -1) # 1, emb_size
# out = projection(emb_sum)

In [52]:
emb_sum
emb_sum.view(1,-1)

tensor([-0.1565,  0.4237, -0.3210], grad_fn=<SumBackward1>)

tensor([[-0.1565,  0.4237, -0.3210]], grad_fn=<ViewBackward>)

In [55]:
projection(emb_sum)
projection(emb_sum.view(1,-1))

tensor([-0.5090,  0.0828, -0.1603, -0.4850,  0.3798, -0.2147,  0.4598, -0.2401,
        -0.5520, -0.3378], grad_fn=<AddBackward0>)

tensor([[-0.5090,  0.0828, -0.1603, -0.4850,  0.3798, -0.2147,  0.4598, -0.2401,
         -0.5520, -0.3378]], grad_fn=<AddmmBackward>)

In [48]:
emb_sum.shape
projection.weight.shape

torch.Size([2, 3])

torch.Size([10, 3])

In [ ]:
class cbow(torch.nn.Module):
    def __init__(self, nwords, emb_size):
        super(cbow, self).__init__()
        #input matrix
        self.emb = torch.nn.Embedding(nwords, emb_size)
        torch.nn.init.uniform_(self.emb.weight, -0.25, 0.25)
        #output matrix
        self.projection = torch.nn.Linear(emb_size, nwords)
        torch.nn.init.uniform_(self.projection.weight, -0.25, 0.25)

    def forward(self, words):
        emb = self.emb(words)
        emb_sum = torch.sum(emb, dim=0) # emb_size
        emb_sum = emb_sum.view(1, -1) # 1, emb_size
        out = self.projection(emb_sum)
        return out

class skip(torch.nn.Module):
    def __init__(self, nwords, emb_size):
        super(skip, self).__init__()
        #input matrix
        self.emb = torch.nn.Embedding(nwords, emb_size)
        torch.nn.init.uniform_(self.emb.weight, -0.25, 0.25)
        #output matrix
        # self.context_emb = torch.nn.Parameter(torch.randn(emb_size, nwords))
        self.context_emb = torch.empty(emb_size, nwords, requires_grad=True).type(torch.cuda.FloatTensor)

    def forward(self, word):
        emb = self.emb(word) # 1, emb_size
        out = torch.matmul(emb, self.context_emb) # 1, nwords
        return out
    

In [ ]:
embeddings_path = "embeddings.txt"

#saving embedding files
with open(embeddings_path, 'w') as embeddings_file:
    W = model.emb.weight.data.cpu().numpy()
    for i in range(nword):
        ith_embedding = '\t'.join(map(str, W[i]))
        embeddings_file.write(ith_embedding + '\n')